# **6.8701 | 6.8710 | HST.507**

#**Fall 2022 Problem Set 5: Clustering Phylogenetic Trees**

Due: Friday, November 19 at 11:59PM (submit copy of notebook to Stellar)

#Load libraries


Feel free to import any additional libraries here.

In [ ]:
from google.colab import files
!pip install ete3
from ete3 import Tree

from sklearn.cluster import AgglomerativeClustering
import numpy as np
from itertools import combinations 



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Clustering Phylogenetic Trees

We've seen that phylogenetic tree algorithms can construct *many* different "good" putative evolutionary histories. An important and challenging problem is that of reducing a large number of trees to a smaller number of representative solutions. The objective of this assignment is to explore techniques for dealing
with large sets of different phylogenetic trees for the same data. This assignment is inspired by the paper [Statistically based postprocessing of phylogenetic analysis by clustering](https://www.ncbi.nlm.nih.gov/pubmed/12169558.) by Cara Stockham, Li-San Wang, and Tandy Warnow (Bioinformatics, Vol 18, Suppl. 1, 2002, pp. S285-S293). This problem is due
to Ran Libeskind-Hadas. The full details are available at the URL below:

http://www.cs.hmc.edu/~hadas/mitcompbio/treedistance.html

You will need to download the following [files](https://www.dropbox.com/sh/94bcsf8hxer11fq/AABqiZjDqHgMHbwJhor7hR5Ja?dl=0) and upload them to colab using the following code block.

In [ ]:
caesal = files.upload()
from Caesal import *

Saving Caesal.py to Caesal.py


A) Implement the *Robinson-Foulds* distance metric.

In [ ]:
"""
Returns distance between two phylogenetic trees

Arguments:
    tree1 - first list of lists or tuple of tuples, with integer member elements
    tree2 - second list of lists or tuple of tuples, with integer member elements
Returns:
    dist - integer distance between 'tree1' and 'tree2'
    """
def distance(tree1, tree2):
    ### YOUR CODE HERE ###
    t1=Tree(str(tree1)+';')
    t2=Tree(str(tree2)+';')
    dist=int(t1.robinson_foulds(t2)[0]/2)
    return dist

These are the test cases for your `distance` function.

In [ ]:
def test_distance():
    T1 = (0, (1, (2, (3, 4))))
    T2 = (0, (1, (3, (2, 4))))
    print("Expected: 1\tOutput: " + str(distance(T1, T2)))
    T3 = (0, ((1, (2, 3)), (7, (6, (4, 5)))))
    T4 = (0, ((2, (1, 3)), (6, (4, (5, 7)))))
    print("Expected: 3\tOutput: " + str(distance(T3, T4)))
    print("Expected: 8\tOutput: " + str(distance(treeList[0], treeList[42])))
    print("Expected: 3\tOutput: " + str(distance(treeList[57], treeList[77])))
    print("Expected: 1\tOutput: " + str(distance(treeList[192], treeList[200])))
    print("Expected: 4\tOutput: " + str(distance(treeList[386], treeList[234])))

In [ ]:
test_distance()

Expected: 1	Output: 1
Expected: 3	Output: 3
Expected: 8	Output: 8
Expected: 3	Output: 3
Expected: 1	Output: 1
Expected: 4	Output: 4


B) You will use a clustering algorithm to partition distinct phylogenetic trees into clusters of trees where the trees in a given cluster are similar with respect to *Robinson-Foulds* distance. (Hint: take note of their hint for speeding up the *Robinson-Foulds* distance calculations!)

In [ ]:
"""
Return each clustering step from 'maxClusters' to 'minClusters' size from input 'treeList'

Arguments:
    treeList - list of all trees to cluster
    minClusters - integer size of minimum number of clusters
    maxClusters - integer size of maximum number of clusters
Returns:
    clusterings - list of clusters with information on numbers of trees, diameters (Robinson-Foulds distance), and average diameter
        [[[numbers of trees], [diameters], average diameter],   <- max number of clusters
         .
         .
         .
         [[numbers of trees], [diameters], average diameter]]   <- min number of clusters
"""
def cluster(treeList, minClusters, maxClusters):
    ### YOUR CODE HERE ###
    #generate 2D distance
    n=len(treeList)
    N=np.zeros((n,n))
    for k in range(1,n):#calculating distance matrix
      for j in range(k,n):
        i=j-k
        N[i,j]=distance(treeList[i],treeList[j])
        N[j,i]=N[i,j]
    X = np.array(N)
    clusterings=[]
    for l in range(maxClusters,minClusters,-1): #for varying number of clusters
          clust = AgglomerativeClustering(n_clusters=l,linkage='average').fit(X) #cluster distance
          clustl=clust.labels_
          numtrees=[]
          diameter=[]
          avgdiam=0
          for m in range(l): #for every cluster
            clustindex=[]
            for o in range(len(clustl)): #keep track of index of tree for its cluster
              if clustl[o]==m:
                clustindex.append(o)
            numtrees.append(len(clustindex)) #number of trees in each clusters
            dist_for_eachC=[]
            pwc=list(combinations(clustindex,2)) #pairwise combinations
            for p in pwc:
              dist_for_eachC.append(N[p]) #retriving distance for pariwise combination
            diameter.append(max(dist_for_eachC)) #max among distance 
            avgdiam=np.average(diameter) #average of the max distances 
          clusterings.append([numtrees,diameter,avgdiam]) #putting things together

    return clusterings

These are the test cases for your `cluster` function.

In [ ]:
def test_cluster():
    clusters = [[[3, 6, 3, 6, 2], [1, 2, 1, 2, 1], 1.4],
                [[9, 3, 6, 2], [2, 1, 2, 1], 1.5],
                [[9, 9, 2], [2, 3, 1], 2],
                [[18, 2], [5, 1], 3],
                [[20], [7], 7]]
    for i, clust in enumerate(cluster(treeList[:20], 1, 5)):
        print("Expected: " + str(clusters[i]) + "\nOutput:   " + str(clust))

In [ ]:
test_cluster()

Expected: [[3, 6, 3, 6, 2], [1, 2, 1, 2, 1], 1.4]
Output:   [[6, 3, 6, 2, 3], [2.0, 1.0, 2.0, 1.0, 1.0], 1.4]
Expected: [[9, 3, 6, 2], [2, 1, 2, 1], 1.5]
Output:   [[9, 3, 6, 2], [2.0, 1.0, 2.0, 1.0], 1.5]
Expected: [[9, 9, 2], [2, 3, 1], 2]
Output:   [[9, 2, 9], [3.0, 1.0, 2.0], 2.0]
Expected: [[18, 2], [5, 1], 3]
Output:   [[18, 2], [5.0, 1.0], 3.0]


/usr/local/lib/python3.8/dist-packages/sklearn/cluster/_agglomerative.py:542: ClusterWarning: scipy.cluster: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  out = hierarchy.linkage(X, method=linkage, metric=affinity)
/usr/local/lib/python3.8/dist-packages/sklearn/cluster/_agglomerative.py:542: ClusterWarning: scipy.cluster: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  out = hierarchy.linkage(X, method=linkage, metric=affinity)
/usr/local/lib/python3.8/dist-packages/sklearn/cluster/_agglomerative.py:542: ClusterWarning: scipy.cluster: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  out = hierarchy.linkage(X, method=linkage, metric=affinity)
/usr/local/lib/python3.8/dist-packages/sklearn/cluster/_agglomerative.py:542: ClusterWarning: scipy.cluster: The symmetric non-negative hollow observation matrix l

C) You will implement a general consensus algorithm that will allow you to find a consensus tree for each cluster and measure the quality of the cluster by its "specificity" (how close it is to being a binary tree).

In [ ]:
"""
Calculate the specificity of 'cluster' given a target 'threshold'

Arguments:
    cluster - list of trees
    threshold - float value of threshold
Returns:
    spec - float specificity of 'cluster' given 'threshold'
"""
def specificity(cluster, threshold):
    ### YOUR CODE HERE ###
    return spec

These are the test cases for your `specificity` function.

In [ ]:
def test_specificity():
    print("Expected: 0.77083\tOutput: " + str(specificity(treeList, 1.0)))
    print("Expected: 0.91667\tOutput: " + str(specificity(treeList, 0.51)))

In [ ]:
test_specificity()

# (6.878 only) Coalescent simulation

In this problem, we will simulate the coalescent process. Recall that this is the time-reversal of the Wright-Fisher process.

A) Write a program to simulate the coalescent process on a population of $2N$ alleles. Track the times of coalescent events starting from the initial generation until all alleles coalesce to a single ancestor. If we are tracking $k$ lineages, you should report $k - 1$ coalescent events.

Recall that the Wright-Fisher process assumes each allele in the next generation is sampled independently from all alleles in the current generation. We are now interested in the reverse, so we instead need to sample parents in the previous generation uniformly at random with replacement. Note that we are interested in the identities of the parents and not their ancestral alleles.

Run 1,000 trials with a population size of $N = 500$. Report the mean and standard deviation of the number of generations between coalescent events for $k = 2$, $3$, $4$ and $5$ lineages.

In [ ]:
### YOUR CODE HERE ###

**Answer here:**

|Lineages|Mean|Standard Deviation|
|-|-|-|
|2|||
|3|||
|4|||
|5|||

B) Recall the waiting time between coalescent events is approximately exponentially distributed with parameter $\lambda$. For each value of $k$, what is the value of $\lambda$ given $N = 500$?

Assuming this distribution, the mean waiting time and its standard deviation are both $\frac{1}{\lambda}$. How do these expected values compare to your observed values? If your observed values are different, give an explanation of what could have caused the differences.

**Answer here:**

|Lineages|$\lambda$|$\frac{1}{\lambda}$|
|-|-|-|
|2|||
|3|||
|4|||
|5|||

C) Extend your simulator to model sexual reproduction.

Assume a fixed number of females $F$ (and therefore $M = N - F$ males) in each generation and that each chromosome in the next generation is selected in the following way: sample a male and female to mate uniformly at random, then sample one of the two alleles uniformly at random. Your simulation should do
the reverse: sample a father and mother and then pick one at random as the ancestor for each allele.

Run 1,000 trials with $F = 100$ and $M = 400$. Do your results agree with the coalescent approximation? Justify your results.

Can you extend the coalescent approximation to more accurately reflect this model of sexual reproduction? Do your results agree with this new approximation?

In [ ]:
### YOUR CODE HERE ###

**Answer here:**

|Lineages|Mean|Standard Deviation|
|-|-|-|
|2|||
|3|||
|4|||
|5|||